# Abstract

In [ ]:
import shap
import pandas as pd

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt


from tqdm.notebook import tqdm

from IPython.display import YouTubeVideo

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedKFold, train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

In [ ]:
raw_data  = pd.read_csv('../input/divorce-prediction/divorce_data.csv', delimiter=';')
reference = pd.read_csv('../input/divorce-prediction/reference.tsv', delimiter='|')

In [ ]:
def get_reference(i, verbose = True):
    question = reference.loc[i,'description']
    if verbose:
        print('Q{}: {}'.format(i,question))
    else:
        return(question)

Our dataset has no missing values

In [ ]:
# Check missingness
raw_data.isnull().any().sum()

In [ ]:
# Plot correlogram
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
plt.figure(figsize=(20,16))
sns.heatmap(raw_data.corr(), cmap='viridis')
plt.show()

In [ ]:
def evaluate_model(preds, test_y, verbose = True, threshold = 0.5):
    
    preds_int = preds >= threshold
    
    
    accuracy = accuracy_score(test_y, preds_int)
    roc_auc  = roc_auc_score(test_y, preds)
    pr_auc   = average_precision_score(test_y, preds)
    f1_val   = f1_score(test_y, preds_int)
    
    if verbose:
        print('Accuracy: {}'.format(accuracy))
        print('AUROC:    {}'.format(roc_auc))
        print('PRAUC:    {}'.format(pr_auc))
        print('F1 Score: {}'.format(f1_val))
    
    
    results = [accuracy, roc_auc, pr_auc, f1_val]
    return(results)

In [ ]:
def run_experiment(dataframe, n = 100, use_tqdm=True):
    results = pd.DataFrame()
    
    if use_tqdm:
        iterator = tqdm(range(n))
    else:
        iterator = range(n)
    
    for i in iterator:
        train_x, test_x = train_test_split(dataframe,
                                   test_size = 0.3,
                                   random_state = i
                                  )
        
        
        train_y = train_x.pop('Divorce')
        test_y  = test_x.pop('Divorce')
        
        rf_model = RandomForestClassifier()
        rf_model.fit(train_x, train_y)
        
        preds    = rf_model.predict_proba(test_x)[:,1]
        
        current_results = evaluate_model(preds, test_y, verbose = False)
        results = results.append([current_results])
    
    
    results.reset_index(drop=True, inplace=True)
    results.columns = ['accuracy','roc_auc','pr_auc','f1_score']
    return(results, rf_model)

In [ ]:
rf_results, rf_model = run_experiment(raw_data)

# Model Evaluation

Our RandomForest performed really, really well, even without tunning. And such performance is consistent as well, as it can be seen from the plots bellow.

In [ ]:
print('......................................')
print('Experiment results (mean of 100 runs)')
print('......................................')
print('Accuracy: {}'.format(rf_results.accuracy.mean()))
print('AUROC:    {}'.format(rf_results.roc_auc.mean()))
print('PRAUC:    {}'.format(rf_results.pr_auc.mean()))
print('F1 Score: {}'.format(rf_results.f1_score.mean()))

In [ ]:
fig = px.box(rf_results.melt(var_name='metric'),
               x = 'metric',
               y = 'value',
               color_discrete_sequence= ['#fc0362'],
               title = 'Distribution of Metric Values Across 100 Runs',
               template = 'plotly_dark'
              )


fig.update_xaxes(title='Metric', gridcolor = 'rgba(240,240,240, 0.05)')
fig.update_yaxes(title='Value', gridcolor = 'rgba(240,240,240, 0.05)')

fig.update_layout({'plot_bgcolor': 'rgba(40, 40, 40, 1.0)',
                   'paper_bgcolor': 'rgba(30, 30, 30, 1.0)',
                  })
fig.show()

Now that we have a pretty decent model, lets use ML interpretability in order to draw knowledge about relationships.

# Foreword on Relationship Lessons

Before any attempt to extract lessons, it is worth keeping some things in mind: Take things with a pinch of salt, as the data was collected in various regions of Turkey, and regional and cultural differences do play a role in the notions of what makes a relationship.

Also, the researchers stated a selection bias for either picking very happy couples or already divorced ones, so we should expect to see great results as the harder-to-predict cases were intentionally left out.

Also, it is worth nothing that research on the topic of workforce motivation differentiates factors that prevent disatisfaction from factors that create satisfaction. That is to say, for example, having a safe workplace might hamper demotivation, however adding more and more safety past a threshold will never make it a great place to work on its own.

The same could apply to relationships, the absense of divorce factors does not create a satisfying relationship on its own. For more information on such research, please refer to the video bellow.

Finally we are drawing conclusions from a single type of relationship (marriage) and trying to extrapolate it to other kinds of relationships.
If in one hand marriages are more comprehensive than other forms of relationships and could therefore provide a better and more complete picture of what makes great relationships, it could also point to factors that may not be as relevant for other forms of relationships.


In [ ]:
# Herzberg two factor theory of motivation
YouTubeVideo('f-qbGAvR4EU', width=800, height=450)

# Relationship Lessons

This dataset is based on the research conducted by [Dr. Gottman](https://en.wikipedia.org/wiki/John_Gottman) and the questions essentially distill his divorce predictors.

These divorce predictors are also known as the **four horsemen of apocalypse** as they are the major themes in the context of divorce.
The four horsemen are:
- Criticism
- Contempt
- Defensiveness
- Stonewalling

The video bellow gives a quick overview of these predictors.

In [ ]:
# Gottham four horsemen of apocalipse
YouTubeVideo('1o30Ps-_8is', width=800, height=450)

It is one thing to grasp the general concepts, but it is another to recognize them on a day to day basis.

We will start by interpreting the most important features (survey questions) on an individual basis and after that we will look into their associations, as in the complexity of a relationship it will be rare to find a scenario as simples as a single feature.

In [ ]:
explainer   = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(raw_data)

In [ ]:
# Average feature contribution
plt.title('Average Feature Contribution for each Class')
shap.summary_plot(shap_values, raw_data, plot_type="bar", plot_size = (15,10))

# Keep top 20 most important feature indexes for later
question_list = [17,19,5,20,18,40,39,9,25,11,15,27,38,26,4,3,41,36,14,22]

In [ ]:
# Granular feature contribution plot
plt.title('Feature Contribution According to Value')
shap.summary_plot(shap_values[0], raw_data, plot_size = (15,10))

### How to read this plot
> SHAP values make use of Shapley equations to frame the prediction as a cooperative game - where are all features must work together in order to achieve a result great than the sum of its individual parts.

> The color represents the feature value (scale from 0 = never to 4 = always in this dataset) and the X axis represents the impact on the final decision (divorce is to right).

In [ ]:
# Question outline of 20 most important features
for question_index in question_list:
    get_reference(question_index)

# Feature Association

In this section we will uncover associations among the questions and draw possible scenarios that these questions could represent, for the sake of further understanding when and how these predictors materialize.

Relationships are complex by nature, marriages specially, so it should be rare to have a single sole predictor being the cause of the divorce.
As years of empirical experience on the field of flight risks, it was noted that major problems are never due to a single root cause, but rather from multiple major causes.

Much like a plane falling from the sky, a relationship is likely to fall due to multiple horsemen being nourished from the patterns of behaviour - and now We'll try to uncover such patterns by looking at feature associations.



### Avoidance and Lack of Communication
From the association of questions 5 and 36 we can picture a scenario where the partners have few time for their relationship and often discuss in a non calm manner.
It is a common pattern to "escape to work" or other activities in order to diminish the extent and therefore impact of a unsatisfying relationship.

At the same time, having fewer time as partners may stack insatisfactions that will "burst" during a conversation, causing it to not be calm.


This association materializes both Stonewalling and Criticism.

In [ ]:
shap.dependence_plot("Q5", shap_values[1], raw_data, show=False)
plt.title('Dependance Plot: Q5 and Q36')
plt.show()

get_reference(5)
get_reference(36)

In [ ]:
for i in range(1,55):
    shap.dependence_plot("Q{}".format(i), shap_values[1], raw_data)

This is a work in progress, I'm preparing a full length notebook, covering all 4 of his books as well as this dataset!